In [4]:
#!/usr/bin/python3

"""
从SSR订阅链接获得ssr服务器，随机选择一个服务器作为代理启动。
"""


import requests
import base64
import os
from random import randint
import re

ssr_tmp = '/tmp/ssr_tmp'

# 订阅链接
ssr_links = requests.get("https://doubledou.win/link/xBpI79VrpV27IOZB?mu=0").text
# print(ssr_links)

class ssr_config():
    """
    SSR配置
    """
        
    def __init__(self,server,port,password,method,remarks):
        self.server = server
        self.port = port
        self.password = password
        self.method = method
        self.remarks = remarks
    
    def command(self):
        """
        生成启动代理服务器命令
        """
        print(f'Current Proxy Address : {self.server}')
        return f'sslocal -s {self.server} -p {self.port} -b 127.0.0.1 -l 1080 -k {self.password} -m {self.method}'

    

def base64_decode(s):
    """
    采用base64 解密
    """
    i = len(s) % 4
    if i == 1 :
        s = s + '='
    elif i == 2 :
        s = s + '=='
    elif i == 3 :
        s = s + '==='

    return str(base64.urlsafe_b64decode(s), encoding = "UTF-8")

ssr_list = []

# 解密出ssr链接
#print(ssr_links)
try:
    ssr_links = base64_decode(ssr_links)
    with open(ssr_tmp,'w') as f:
        f.write(ssr_links)
except:
    print("解密失败，try 缓存订阅")
    try:
        with open(ssr_tmp,'r') as f:   
            ssr_links = f.read()
    except FileNotFoundError as e:
        print("缓存文件不存在，无法解析")
        exit()

# 对每个ssr链接解密
for line in ssr_links.split('\n'):
    if line == '':
        continue
    try:
        ssr_list.append(base64_decode(line.replace('ssr://','')))
    except:
        continue

ssr_config_list = []        

def build_condig(s):
    """
    从文本中解析生成 ssr_config 对象
    """
    info,info2 = s.split('/?')
    detail = info.split(':')
    remarks = re.search('remarks=.+?\&',info2).group()[:-1].split('=')[1]
    # print(detail)
    return ssr_config(detail[0],detail[1],base64_decode(detail[-1]),detail[3],base64_decode(remarks))

# 生成ssr配置对象列表
for item in ssr_list:
    # print(item)
    ssr_config_list.append(build_condig(item))

print("****启动代理****")
for i,v in enumerate(ssr_config_list):
    print(f'{i:>3} : {v.remarks}')

print("****"*4)

c = -1

while c < 0 or c >= len(ssr_config_list):
    c = int(input('输入代理序号：'))

print(f'启动：{ssr_config_list[c].remarks}')
os.system(ssr_config_list[c].command())

KeyboardInterrupt: 